# Bias-variance trade-off, model selection and cross validation – Computer exercises

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.preprocessing as skl_pre
import sklearn.linear_model as skl_lm
import sklearn.discriminant_analysis as skl_da
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms

## 6.1 Cross validation in $k$-NN
In this exercise we will return to the `biopsy` data set also used in Exercise 4.1 (Lesson 4). We will try to determine suitable value of $k$ in $k$-NN for this data. For simplicity, we will only consider the three attributes in columns `V3`, `V4`and `V5` in this problem.

### (a)
Consider all data as training data. Investigate how the training error varies with different values of $k$ (hint: use a `for`-loop). Which $k$ gives the best result? Is it a good choice of $k$?

In [40]:
# Load the data
# url = 'data/biopsy.csv'
url = 'https://uu-sml.github.io/course-sml-public/data/biopsy.csv'
biopsy = pd.read_csv(url, dtype={'ID': str}).dropna().reset_index(drop=True)

biopsy.head()

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,class
0,1000025,5,1,1,1,2,1.0,3,1,1,benign
1,1002945,5,4,4,5,7,10.0,3,2,1,benign
2,1015425,3,1,1,1,2,2.0,3,1,1,benign
3,1016277,6,8,8,1,3,4.0,3,7,1,benign
4,1017023,4,1,1,3,2,1.0,3,1,1,benign


In [73]:

import sklearn.metrics as skl_me
X = biopsy[['V3', 'V4', 'V5']]
Y = biopsy['class'].map({'benign': 0, 'malignant': 1})

for k in range(1, 15):
  model = skl_nb.KNeighborsClassifier(n_neighbors=k)
  model.fit(X, Y)
  Y_pred = model.predict(X)
  print(f'MSE for k={k}: {skl_me.accuracy_score(Y, Y_pred)}')

# K=1 gives the best result, but it is not a good choice for generalization




MSE for k=1: 0.9663250366032211
MSE for k=2: 0.9516837481698389
MSE for k=3: 0.9619326500732065
MSE for k=4: 0.9531478770131772
MSE for k=5: 0.9546120058565154
MSE for k=6: 0.9516837481698389
MSE for k=7: 0.9546120058565154
MSE for k=8: 0.9516837481698389
MSE for k=9: 0.9546120058565154
MSE for k=10: 0.9502196193265008
MSE for k=11: 0.9516837481698389
MSE for k=12: 0.9502196193265008
MSE for k=13: 0.9516837481698389
MSE for k=14: 0.9516837481698389


### (b) <a id='6.1-b'></a>
Split the data randomly into a training and validation set, and see how well you perform on the validation set. (Previously, we have used the terminology "training" and "test" set. If the other set (not the training set) is used to make design decisions, such as choosing $k$, it is really not a test set, but rather a "validation" set. Hence the
terminology.) Which $k$ gives the best result?

In [66]:
X_train, X_validation, Y_train, Y_validation = skl_ms.train_test_split(X, Y, test_size=0.3, random_state=1)

for k in range(1, 201):
  model = skl_nb.KNeighborsClassifier(n_neighbors=k)
  model.fit(X_train, Y_train)
  Y_pred = model.predict(X_validation)
  print(f'Accuracy score for k={k}: {skl_me.accuracy_score(Y_validation, Y_pred)}')

# k = 3 and k=5 is the best


Accuracy score for k=1: 0.9317073170731708
Accuracy score for k=2: 0.926829268292683
Accuracy score for k=3: 0.9609756097560975
Accuracy score for k=4: 0.9560975609756097
Accuracy score for k=5: 0.9609756097560975
Accuracy score for k=6: 0.9560975609756097
Accuracy score for k=7: 0.9560975609756097
Accuracy score for k=8: 0.9560975609756097
Accuracy score for k=9: 0.9560975609756097


### (c)
Perform [(b)](#6.1-b) 10 times for different validation sets and average the result. Which $k$ gives the best result?

In [83]:
number_of_ks = 40
average = [0] * number_of_ks
for i in range(number_of_ks):
    X_train, X_validation, Y_train, Y_validation = skl_ms.train_test_split(
        X, Y, test_size=0.3, random_state=i
    )

    for k in range(1, number_of_ks):
        model = skl_nb.KNeighborsClassifier(n_neighbors=k)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_validation)
        average[k] += skl_me.accuracy_score(Y_validation, Y_pred)

average = [x / number_of_ks for x in average]
best_k = 1 + average[1:].index(max(average[1:]))
print("Average accuracies:", average)
print("Best k:", best_k)


Average accuracies: [0.0, 0.9264634146341463, 0.9178048780487806, 0.9469512195121954, 0.9417073170731711, 0.9510975609756093, 0.9479268292682926, 0.9507317073170732, 0.9496341463414637, 0.9506097560975609, 0.9496341463414634, 0.9499999999999998, 0.9491463414634147, 0.9496341463414634, 0.9490243902439024, 0.9496341463414633, 0.9491463414634147, 0.9493902439024389, 0.9481707317073171, 0.9489024390243902, 0.9480487804878048, 0.9490243902439024, 0.9484146341463415, 0.9486585365853658, 0.9490243902439026, 0.9492682926829268, 0.9503658536585367, 0.9507317073170729, 0.9503658536585362, 0.9509756097560974, 0.9507317073170729, 0.9506097560975608, 0.9504878048780485, 0.9508536585365853, 0.9514634146341461, 0.9512195121951217, 0.9510975609756095, 0.9512195121951217, 0.9507317073170732, 0.9508536585365853]
Best k: 34


### (d)
Perform 10-fold cross-validation by first randomly permute the data set, divide the data set into 10 equally sized parts and loop through them by taking one part as validation set and the rest as training set each time. Which $k$ gives the best result?

In [84]:
n_fold = 10
skl_ms.KFold(n_splits=10, shuffle=True)



KFold(n_splits=10, random_state=None, shuffle=True)

## 6.2 Cross validation for model choice

In this problem we will consider the data sets `data/pima_tr.csv` and `data/pima_te.csv`. Your task is to do as good prediction as possible for the test set `pima_te`, but you are only allowed to look at the true output in `pima_te` once (like in the real life, where you design and implement a method, and then hand it over to the ultimate test, namely the user). Hence, you will have to use `pima_tr` for both deciding which model to use and training the model.

The data set describes the prevalence of diabetes in women at least 21 years old of Pima Indian heritage, living near Phoenix, Arizona, USA. The data set describes, for each individual, whether she has diabetes or not, her age, the diabetes pedigree function (a summary of the diabetes history in her family), BMI, skin thickness, blood pressure, plasma glucose concentration and number of pregnancies.

The data frame contains the following columns:  
`npreg` number of pregnancies.  
`glu` plasma glucose concentration in an oral glucose tolerance test.  
`bp` diastolic blood pressure (mm Hg).  
`skin` triceps skin fold thickness (mm).  
`bmi` body mass index (weight in kg/(height in m)\^2).  
`ped` diabetes pedigree function.  
`age` age in years.  
`type` Yes or No, for diabetic according to WHO criteria.


### (a)
Load the library and familiarize yourself with `pima_tr`

In [ ]:
# Load the datasets
# url = 'data/pima_tr.csv'
url = 'https://uu-sml.github.io/course-sml-public/data/pima_tr.csv'
pima_tr = pd.read_csv(url)

# url = 'data/pima_tr.csv'
url = 'https://uu-sml.github.io/course-sml-public/data/pima_te.csv'
pima_te = pd.read_csv(url)

### (b)
See how well you can fit the `pima_tr` with logistic regression, LDA, QDA and k-NN (k = 2). The output is whether an individual has diabetes or not, and the input the remaining variables. What error rate does each method have? Is it a good indicator of which method is preferable?

### (c)
Instead of [(b)](#6.2-b), perform 10-fold cross-validation by first randomly permute `pima_tr` and divide it in 10 parts. Then, in a loop with one of the 10 parts held out as validation data, fit logistic regression, LDA, QDA and k-NN (k = 2) to the training data and evaluate the performance on the validation data. Plot your results in a box plot with the error rates. Feel free to play around with the choice of inputs and other settings to improve the performance. Which method does this suggest us to use?

### d) <a id='6.4-d'/>
Now, decide which method to choose and train it on the entire data set `pima_tr` and predict `pima_te`. How well do you perform?

### (e)
Now, since we are in a simulated environment, we can cheat and break the rule that we were only allowed to look at the true output in `pima_te` once. That is, explore how well the other methods do when you train them on `pima_tr` and predict `pima_te`. Did you make the "right" choice in [(d)](#6.4-d)?

## 6.3 Implementing problem 5.3
Verify your theoretical findings from problem 5.3 by repeating the experiment $N$ times and approximating all expected values with sums. Let $\sigma^2=1$.

### a)
Generate training data ($n=1$), estimate $\theta_0$ and compute $\widehat y(x_\star;\mathcal{T})$. Repeat $N$ times and store the results in a vector. Choose the regularization parameter yourself.

### b)
Estimate $\bar{f}(x_\star)=\mathbb{E}_\mathcal{T}[y(x_\star;\mathcal{T})]$ from your vector of $\widehat y(x_\star,\mathcal{T})$. Compare your result to your theoretical findings in 5.3b.

### c)
Estimate the square bias $\mathbb{E}_\star[(\bar{f}(x_\star)-f_0(x_\star))^2]$ using your result from b) and your knowledge about the true $f_0(x)$. Compare your result to your theoretical findings in 5.3c.

### d)
Estimate the variance $\mathbb{E}_\star[\mathbb{E}_\mathcal{T}[(\widehat y (x_\star;\mathcal{T}) - \bar f(x_\star))^2]]$ using your vector of $\widehat y(x_\star;\mathcal{T})$ from a) and your result from b). Compare your result to your theoretical findings in 5.3d.

### e)
Estimate the expected new data error $\bar E_\text{new} = \mathbb{E}_\mathcal{T}[E_\text{new}] = \mathbb{E}_\mathcal{T}[\mathbb{E}_\star[(y(x_\star;\mathcal{T})-\bar{f}(x_\star))^2]]$ by, for each $\widehat y(x_\star;\mathcal{T})$ in your vector from a), simulate $N$ copies of $y_\star$. Compare your result to your theoretical findings in 5.3f.

### f)
Make a loop over different values for the regularization parameter $\lambda$ and plot bias, variance and $\bar{E}_\text{new}$as a function of $\lambda$. Also plot your theoretical findings from 5.3 in the same plot.

## 6.4 Implementing problem 5.5

Design an experiment (similarly to 6.3) where you numerically confirm the results from problem 5.5.